1. 인스타그램 화면구성 / 망원동 검색결과
3. 크롤링 
    1. 데이터수집과정
    2. 데이터구조
        * 아이디,사진, 날짜, 댓글, 글,  태그, 위치 등
    3. 위치정보
4. 위치정보 정리
    1. 위도 경도 필요함
    2. 가게이름 ☞ 상호명, 주소  by 네이버지도API
    3. 주소☞ 위.경도 데이터 by 네이버API
    4. 포괄적 정보 제외 ex 망원동, 서울.....
5. 지도 시각화
6. 데이터 해석 / Insight

# 1. 인스타그램

https://www.instagram.com/explore/tags/망원동/

- 게시글에 위치정보 포함

- 추천 위치 선택 or 직접 입력 가능

- 인스타 위치정보 ==> 분석 / 시각화 활용?

# 2. 크롤링

 - 코드(함수별)

In [ ]:
# selenium 불러오기
from selenium import webdriver
import re
from bs4 import BeautifulSoup
import json
import time

In [ ]:
# webdriver 열어주기
# webdriver.Chrome('파일경로')
def open_browser():
    browser = webdriver.Chrome('C:/informs/chromedriver')
    return browser

In [ ]:
def insta_searching(browser):
    word = input('인스타 검색어를 입력해주세요? ')
    url = 'https://www.instagram.com/explore/tags/{}'.format(word)
    browser.get(url)

In [ ]:
# 첫번째 검색결과 클릭
def select_first(browser):
    browser.find_element_by_xpath('//*[@id="react-root"]/section/main/article/div[1]/div/div/div[1]/div[1]/a/div/div[2]').click()


In [ ]:
def get_author(browser):
    author = browser.find_element_by_css_selector('div.C4VMK > h2 > a').text
    return author

In [ ]:
# 본문 전체 가져오기
def get_content(browser):
    content = browser.find_element_by_css_selector('div.C4VMK').text
    author = get_author(browser)
    content = content[len(author):]
#     body > div:nth-child(13) > div > div._o0j5z > div > article > div._ebcx9 > div._4a48i._277v9 > ul._b0tqa > li._ezgzd
#     print(content)
    return content


In [ ]:
# 본문의 태그를 리스트 형태로 저장해줌(정규식 사용)
# 정규식 사용 관련 참고: http://www.nextree.co.kr/p4327/
# 정규식표현기 : https://regex101.com/#python
# 정규식 파이썬 메뉴얼 : https://docs.python.org/2/library/re.html
def get_tags(browser):
    content = get_content(browser)
    tags = re.findall(r'#[^\s#,\\]+', content)  #태그만 걸러줌
    # tags = re.findall(r'#[\S][^#\\]+', content)
    # tags = re.findall(r'#[^#\s\',\(\)!\.~\-/&\+\*$]+', content)
    return tags

In [ ]:
def get_location(browser):
    try:
        location = browser.find_element_by_css_selector('a.O4GlU').text
    except:
        return None
    return location

In [ ]:
# 다음 게시글로 이동
def move_to_next(browser):
    move_next = browser.find_element_by_css_selector('a.HBoOv.coreSpriteRightPaginationArrow')
    move_next.click()
    browser.implicitly_wait(3)
    time.sleep(1)

In [ ]:
def get_datetime(browser):
    datetime = browser.find_element_by_css_selector('time._1o9PC.Nzb55').get_attribute('datetime')
    datetime = datetime[:10]
    return datetime
    #2018-10-19T15:00:20.000Z 형태

In [ ]:
def get_img_link(browser):
    img_link = browser.find_element_by_css_selector('div.eLAPa.kPFhm > div.KL4Bh > img.FFVAD').get_attribute('src')
    return img_link

In [ ]:
# 하나의 글에서  본문내용/태그/댓글 모두 가져오기(dictionary 형태)
def get_total_dict(browser):
    datas = {}
    datas['author'] = get_author(browser)
    datas['content'] = get_content(browser)
    datas['tags'] = get_tags(browser)
    datas['location'] = get_location(browser)
    datas['datetime'] = get_datetime(browser)
    datas['img'] = get_img_link(browser)
#     more_reply(browser)
#     datas['reply'] = get_reply(browser)
    return datas

 - 실행 부분

In [ ]:
browser = open_browser()  #크롬 브라우저 실행

In [ ]:
insta_searching(browser)  #인스타 검색: '망원동'

In [ ]:
select_first(browser)  # 첫번째 게시글 클릭

In [ ]:
from tqdm import tqdm_notebook

In [ ]:
#망원동  게시물 694,508 중 15256개 수집
#망리단길 게시물 210,659 중 9056
# 3일 소요......

results={}
for i in tqdm_notebook(range(20000)):
    try:
        results[i] = get_total_dict(browser)
    except:
        pass
    move_to_next(browser)

 - 수집데이터 정리하기

In [4]:
import json
import pandas as pd

In [ ]:
# pandas --> csv 로 저장하기
datas = pd.DataFrame(results)
datas = datas.T
datas.head()
datas.to_excel('망원동_total.xlsx', encoding='utf-8', index=False)

In [5]:
datas = pd.read_excel('망원동_total.xlsx', encoding='utf-8', index_col=0)

In [6]:
datas.head()

,author,content,datetime,img,location,tags
index,,,,,,
0,sunhye1012,#올인원파스타\r\r\n요즘 망원동이 핫하다던데 울신랑님 아는동생도 이쪽에 파스타집...,2018-09-18,https://scontent-icn1-1.cdninstagram.com/vp/c0...,망원동맛집,"['#올인원파스타', '#망원동맛집', '#망리단길', '#망원동파스타', '#파스..."
1,arang2o_o,에휴.. 안줄거 다 안다 😔😥💧,2018-10-19,https://scontent-icn1-1.cdninstagram.com/vp/56...,훌라훌라 Hulahula,[]
2,bbangsuni434,#망원동#소금집#소금집델리 #파스트라미샌드위치#잠봉뵈르#이탈리안히어로\r\r\n(사...,2018-09-18,https://scontent-icn1-1.cdninstagram.com/vp/64...,NaN,"['#망원동', '#소금집', '#소금집델리', '#파스트라미샌드위치', '#잠봉뵈..."
3,luv__jane_,.\r\r\n#망원동 .\r\r\n.\r\r\n퇴사한 언니랑 휴직중 지나:) 이쁜지...,2018-09-18,https://scontent-icn1-1.cdninstagram.com/vp/30...,Cafe Miei,"['#망원동', '#망원동', '#망리단길', '#카페미아이', '#miei', '..."
4,c.blueprint,무료로 스냅사진을 찍어드리는\r\r\n망원동 망리단길 카페 '청사진'\r\r\n'C...,2018-09-18,https://scontent-icn1-1.cdninstagram.com/vp/35...,카페청사진,[]


In [7]:
datas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19652 entries, 0 to 22882
Data columns (total 6 columns):
author      19652 non-null object
content     19651 non-null object
datetime    19652 non-null object
img         17670 non-null object
location    9364 non-null object
tags        19652 non-null object
dtypes: object(6)
memory usage: 1.0+ MB


In [8]:
# location 위치 입력 비율  : 48%
len(datas[datas.location.notnull()]) / len(datas)

0.47649094239772033